# Model 1: Baseline Models - Validation Tests

This notebook validates that all baseline models work correctly and produce sensible predictions.

## Validation Tests

| Test | Description | Pass Criteria |
|------|-------------|---------------|
| 1. Sanity Check | Predictions within valid range | 0 ≤ goals ≤ 15 |
| 2. Consistency | Same input → same output | Deterministic |
| 3. Fit Required | Can't predict before fitting | Raises error |
| 4. Team Distinction | Different teams → different predictions | TeamMean varies |
| 5. Home Advantage | Home teams score more on average | home_goals > away_goals |
| 6. Evaluation | Metrics computed correctly | RMSE > 0 |
| 7. Edge Cases | Handle missing teams gracefully | Uses defaults |

## Setup

In [ ]:
import pandas as pd
import numpy as np
import os
import sys

# Add parent for imports
sys.path.insert(0, os.path.dirname(os.getcwd()))

print("Setup complete.")

In [ ]:
# Self-contained baseline model classes for validation

COLUMN_ALIASES = {
    'home_team': ['home_team', 'home', 'team_home', 'h_team'],
    'away_team': ['away_team', 'away', 'team_away', 'a_team', 'visitor'],
    'home_goals': ['home_goals', 'home_score', 'h_goals', 'goals_home'],
    'away_goals': ['away_goals', 'away_score', 'a_goals', 'goals_away'],
}

def get_value(game, field, default=None):
    aliases = COLUMN_ALIASES.get(field, [field])
    for alias in aliases:
        if alias in game:
            val = game[alias]
            if pd.isna(val):
                return default
            return val
    return default

def get_column(df, field):
    aliases = COLUMN_ALIASES.get(field, [field])
    for alias in aliases:
        if alias in df.columns:
            return alias
    return None


class BaselineModel:
    def __init__(self, params=None):
        self.params = params or {}
        self.is_fitted = False
    
    def evaluate(self, games_df):
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before evaluation")
        from sklearn.metrics import mean_squared_error, mean_absolute_error
        home_preds, home_actuals = [], []
        for _, game in games_df.iterrows():
            home_pred, _ = self.predict_goals(game)
            home_preds.append(home_pred)
            home_actuals.append(get_value(game, 'home_goals', 0))
        rmse = mean_squared_error(home_actuals, home_preds, squared=False)
        mae = mean_absolute_error(home_actuals, home_preds)
        return {'rmse': rmse, 'mae': mae}


class GlobalMeanBaseline(BaselineModel):
    def fit(self, games_df):
        home_col = get_column(games_df, 'home_goals')
        away_col = get_column(games_df, 'away_goals')
        self.global_mean_home = games_df[home_col].mean()
        self.global_mean_away = games_df[away_col].mean()
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        return self.global_mean_home, self.global_mean_away


class TeamMeanBaseline(BaselineModel):
    def fit(self, games_df):
        home_team_col = get_column(games_df, 'home_team')
        away_team_col = get_column(games_df, 'away_team')
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        
        goals_for, goals_against, games_played = {}, {}, {}
        
        for _, game in games_df.iterrows():
            ht, at = game[home_team_col], game[away_team_col]
            hg, ag = game[home_goals_col], game[away_goals_col]
            
            goals_for[ht] = goals_for.get(ht, 0) + hg
            goals_against[ht] = goals_against.get(ht, 0) + ag
            games_played[ht] = games_played.get(ht, 0) + 1
            
            goals_for[at] = goals_for.get(at, 0) + ag
            goals_against[at] = goals_against.get(at, 0) + hg
            games_played[at] = games_played.get(at, 0) + 1
        
        self.team_offense = {t: goals_for[t] / games_played[t] for t in games_played}
        self.team_defense = {t: goals_against[t] / games_played[t] for t in games_played}
        self.global_mean = games_df[home_goals_col].mean()
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        ht = get_value(game, 'home_team')
        at = get_value(game, 'away_team')
        home_off = self.team_offense.get(ht, self.global_mean)
        home_def = self.team_defense.get(ht, self.global_mean)
        away_off = self.team_offense.get(at, self.global_mean)
        away_def = self.team_defense.get(at, self.global_mean)
        return (home_off + away_def) / 2, (away_off + home_def) / 2


class HomeAwayBaseline(BaselineModel):
    def fit(self, games_df):
        home_team_col = get_column(games_df, 'home_team')
        away_team_col = get_column(games_df, 'away_team')
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        
        home_goals_for, home_games = {}, {}
        
        for _, game in games_df.iterrows():
            ht = game[home_team_col]
            hg = game[home_goals_col]
            home_goals_for[ht] = home_goals_for.get(ht, 0) + hg
            home_games[ht] = home_games.get(ht, 0) + 1
        
        self.home_offense = {t: home_goals_for[t]/home_games[t] for t in home_games}
        self.global_mean = games_df[home_goals_col].mean()
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        ht = get_value(game, 'home_team')
        home_pred = self.home_offense.get(ht, self.global_mean)
        return home_pred, self.global_mean


class MovingAverageBaseline(BaselineModel):
    def __init__(self, params=None):
        super().__init__(params)
        self.window = self.params.get('window', 5)
    
    def fit(self, games_df):
        home_team_col = get_column(games_df, 'home_team')
        away_team_col = get_column(games_df, 'away_team')
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        
        self.team_history = {}
        
        for _, game in games_df.iterrows():
            ht, at = game[home_team_col], game[away_team_col]
            hg, ag = game[home_goals_col], game[away_goals_col]
            
            if ht not in self.team_history: self.team_history[ht] = []
            if at not in self.team_history: self.team_history[at] = []
            
            self.team_history[ht].append((hg, ag))
            self.team_history[at].append((ag, hg))
        
        self.global_mean = games_df[home_goals_col].mean()
        self.is_fitted = True
        return self
    
    def _get_recent_avg(self, team):
        if team not in self.team_history or len(self.team_history[team]) == 0:
            return self.global_mean, self.global_mean
        recent = self.team_history[team][-self.window:]
        return np.mean([g[0] for g in recent]), np.mean([g[1] for g in recent])
    
    def predict_goals(self, game):
        ht, at = get_value(game, 'home_team'), get_value(game, 'away_team')
        home_off, home_def = self._get_recent_avg(ht)
        away_off, away_def = self._get_recent_avg(at)
        return (home_off + away_def) / 2, (away_off + home_def) / 2


class WeightedHistoryBaseline(BaselineModel):
    def __init__(self, params=None):
        super().__init__(params)
        self.decay = self.params.get('decay', 0.9)
    
    def fit(self, games_df):
        home_goals_col = get_column(games_df, 'home_goals')
        self.global_mean = games_df[home_goals_col].mean()
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        return self.global_mean, self.global_mean


class PoissonBaseline(BaselineModel):
    def fit(self, games_df):
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        self.league_avg = games_df[home_goals_col].mean()
        self.home_factor = games_df[home_goals_col].mean() / max(games_df[away_goals_col].mean(), 0.01)
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        return self.league_avg * self.home_factor, self.league_avg / self.home_factor


print("Baseline models loaded for validation.")

In [ ]:
# Create test data
np.random.seed(42)

# Strong team scores more, weak team scores less
test_data = pd.DataFrame([
    {'home_team': 'Strong', 'away_team': 'Weak', 'home_goals': 5, 'away_goals': 1},
    {'home_team': 'Strong', 'away_team': 'Average', 'home_goals': 4, 'away_goals': 2},
    {'home_team': 'Average', 'away_team': 'Strong', 'home_goals': 2, 'away_goals': 4},
    {'home_team': 'Weak', 'away_team': 'Strong', 'home_goals': 1, 'away_goals': 5},
    {'home_team': 'Average', 'away_team': 'Weak', 'home_goals': 3, 'away_goals': 2},
    {'home_team': 'Weak', 'away_team': 'Average', 'home_goals': 2, 'away_goals': 3},
    {'home_team': 'Strong', 'away_team': 'Weak', 'home_goals': 6, 'away_goals': 0},
    {'home_team': 'Strong', 'away_team': 'Average', 'home_goals': 4, 'away_goals': 3},
])

print("Test data created:")
test_data

## Test 1: Sanity Check - Predictions Within Valid Range

In [ ]:
def test_sanity_check():
    """Test that all predictions are within valid hockey goal range."""
    models = [
        GlobalMeanBaseline(),
        TeamMeanBaseline(),
        HomeAwayBaseline(),
        MovingAverageBaseline({'window': 3}),
        PoissonBaseline()
    ]
    
    for model in models:
        model.fit(test_data)
        
        for _, game in test_data.iterrows():
            home_pred, away_pred = model.predict_goals(game)
            
            # Goals should be non-negative
            assert home_pred >= 0, f"{model.__class__.__name__}: Negative home goals"
            assert away_pred >= 0, f"{model.__class__.__name__}: Negative away goals"
            
            # Goals should be reasonable (< 15 for hockey)
            assert home_pred <= 15, f"{model.__class__.__name__}: Home goals too high"
            assert away_pred <= 15, f"{model.__class__.__name__}: Away goals too high"
    
    return True

try:
    test_sanity_check()
    print("✅ TEST 1 PASSED: All predictions within valid range [0, 15]")
except AssertionError as e:
    print(f"❌ TEST 1 FAILED: {e}")

## Test 2: Consistency - Same Input Produces Same Output

In [ ]:
def test_consistency():
    """Test that predictions are deterministic."""
    model = TeamMeanBaseline()
    model.fit(test_data)
    
    game = test_data.iloc[0]
    
    # Get prediction multiple times
    pred1 = model.predict_goals(game)
    pred2 = model.predict_goals(game)
    pred3 = model.predict_goals(game)
    
    assert pred1 == pred2 == pred3, "Predictions should be identical"
    return True

try:
    test_consistency()
    print("✅ TEST 2 PASSED: Predictions are consistent (deterministic)")
except AssertionError as e:
    print(f"❌ TEST 2 FAILED: {e}")

## Test 3: Fit Required - Can't Predict Before Fitting

In [ ]:
def test_fit_required():
    """Test that evaluation fails if model not fitted."""
    model = TeamMeanBaseline()
    
    try:
        model.evaluate(test_data)
        return False  # Should have raised error
    except RuntimeError:
        return True  # Expected behavior

try:
    if test_fit_required():
        print("✅ TEST 3 PASSED: Model raises error when not fitted")
    else:
        print("❌ TEST 3 FAILED: No error raised for unfitted model")
except Exception as e:
    print(f"❌ TEST 3 FAILED: {e}")

## Test 4: Team Distinction - Different Teams Get Different Predictions

In [ ]:
def test_team_distinction():
    """Test that TeamMean distinguishes between teams."""
    model = TeamMeanBaseline()
    model.fit(test_data)
    
    # Create games with different matchups
    game1 = pd.Series({'home_team': 'Strong', 'away_team': 'Weak'})
    game2 = pd.Series({'home_team': 'Weak', 'away_team': 'Strong'})
    
    pred1 = model.predict_goals(game1)
    pred2 = model.predict_goals(game2)
    
    # Strong team should be predicted to score more
    assert pred1[0] > pred2[0], "Strong home should score more than Weak home"
    
    return True

try:
    test_team_distinction()
    print("✅ TEST 4 PASSED: Model distinguishes between teams")
except AssertionError as e:
    print(f"❌ TEST 4 FAILED: {e}")

## Test 5: Home Advantage - Home Teams Score More on Average

In [ ]:
def test_home_advantage():
    """Test that GlobalMean captures home advantage in data."""
    model = GlobalMeanBaseline()
    model.fit(test_data)
    
    # Our test data has home advantage
    avg_home = test_data['home_goals'].mean()
    avg_away = test_data['away_goals'].mean()
    
    # Model should capture this
    assert model.global_mean_home == avg_home, "Home mean should match data"
    assert model.global_mean_away == avg_away, "Away mean should match data"
    
    return True

try:
    test_home_advantage()
    print("✅ TEST 5 PASSED: Model captures home/away goal averages correctly")
except AssertionError as e:
    print(f"❌ TEST 5 FAILED: {e}")

## Test 6: Evaluation - Metrics Computed Correctly

In [ ]:
def test_evaluation():
    """Test that evaluation returns valid metrics."""
    model = GlobalMeanBaseline()
    model.fit(test_data)
    
    metrics = model.evaluate(test_data)
    
    # Check metrics exist and are valid
    assert 'rmse' in metrics, "RMSE should be in metrics"
    assert 'mae' in metrics, "MAE should be in metrics"
    assert metrics['rmse'] >= 0, "RMSE should be non-negative"
    assert metrics['mae'] >= 0, "MAE should be non-negative"
    assert metrics['rmse'] >= metrics['mae'], "RMSE should be >= MAE"
    
    return True

try:
    test_evaluation()
    print("✅ TEST 6 PASSED: Evaluation metrics computed correctly")
except AssertionError as e:
    print(f"❌ TEST 6 FAILED: {e}")

## Test 7: Edge Cases - Handle Missing Teams Gracefully

In [ ]:
def test_edge_cases():
    """Test handling of unknown teams."""
    model = TeamMeanBaseline()
    model.fit(test_data)
    
    # Unknown team
    unknown_game = pd.Series({
        'home_team': 'NewTeam',  # Not in training data
        'away_team': 'Strong'
    })
    
    try:
        home_pred, away_pred = model.predict_goals(unknown_game)
        
        # Should use global mean as fallback
        assert home_pred > 0, "Unknown team should get global mean"
        assert away_pred > 0, "Known team should get valid prediction"
        
        return True
    except Exception:
        return False

try:
    if test_edge_cases():
        print("✅ TEST 7 PASSED: Unknown teams handled gracefully with defaults")
    else:
        print("❌ TEST 7 FAILED: Error handling unknown teams")
except Exception as e:
    print(f"❌ TEST 7 FAILED: {e}")

## Test 8: Column Flexibility - Different Column Names

In [ ]:
def test_column_flexibility():
    """Test that models work with different column naming conventions."""
    # Alternative column names
    alt_data = pd.DataFrame([
        {'home': 'A', 'away': 'B', 'home_score': 3, 'away_score': 2},
        {'home': 'B', 'away': 'A', 'home_score': 2, 'away_score': 4},
        {'home': 'A', 'away': 'B', 'home_score': 5, 'away_score': 1},
    ])
    
    model = TeamMeanBaseline()
    model.fit(alt_data)
    
    game = alt_data.iloc[0]
    home_pred, away_pred = model.predict_goals(game)
    
    assert home_pred > 0, "Should work with 'home' column"
    assert away_pred > 0, "Should work with 'away' column"
    
    return True

try:
    test_column_flexibility()
    print("✅ TEST 8 PASSED: Model works with alternative column names")
except Exception as e:
    print(f"❌ TEST 8 FAILED: {e}")

## Summary

In [ ]:
# Run all tests and summarize
tests = [
    ("Sanity Check", test_sanity_check),
    ("Consistency", test_consistency),
    ("Fit Required", test_fit_required),
    ("Team Distinction", test_team_distinction),
    ("Home Advantage", test_home_advantage),
    ("Evaluation", test_evaluation),
    ("Edge Cases", test_edge_cases),
    ("Column Flexibility", test_column_flexibility),
]

print("\n" + "="*50)
print("BASELINE MODEL VALIDATION SUMMARY")
print("="*50 + "\n")

passed = 0
failed = 0

for name, test_func in tests:
    try:
        result = test_func()
        if result:
            print(f"✅ {name}")
            passed += 1
        else:
            print(f"❌ {name}")
            failed += 1
    except Exception as e:
        print(f"❌ {name}: {e}")
        failed += 1

print(f"\n{'='*50}")
print(f"Results: {passed} passed, {failed} failed")
print("="*50)

if failed == 0:
    print("\n ALL TESTS PASSED - Baseline models are validated")
else:
    print(f"\n  {failed} test(s) failed - Review issues above")